# Librerías

In [1]:
from google.colab import drive
import sys
import os
import sqlite3 as sql #Crear y trabajar bases de datos ligeras
import pandas as pd

# Conexión con la BD y el repositorio en Drive

In [4]:
#Conectar al drive local
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Conectar al repositorio
path = '/content/drive/My Drive/cod/A3_marketing'

sys.path.append(path) #Importar las funciones propias a través de import, porque incluye la carpeta del repositorio como uno de esos paquetes para que import busque funciones
os.chdir(path) #Subir y descargar archivos de la ruta del repositorio de trabajo

# Conexión al notebook de funciones

In [6]:
%run a_funciones.ipynb

# SQL

## Conexión

In [7]:
con = sql.connect('Data/db_movies') #Conectarse a la base de datos existente y transportar datos
cur = con.cursor() #Otra conexión (cursor) para ejecutar las consultas en la bd sin traer ni llevar info

In [8]:
#Consultar qué tablas tiene la bd disponibles
cur.execute(""" select name from sqlite_master where type= 'table' """)
cur.fetchall()

[('ratings',), ('movies',)]

## Exploración

In [30]:
tablas_a_explorar = ['ratings', 'movies', 'ratings2']
for tabla in tablas_a_explorar:
    explorar_tabla(con, tabla)

------------

Explorando la tabla 'ratings'

Esquema de la tabla 'ratings':
  - Nombre: userId, Tipo: INTEGER, ¿Nulo?: 0, Clave Primaria: 0
  - Nombre: movieId, Tipo: INTEGER, ¿Nulo?: 0, Clave Primaria: 0
  - Nombre: rating, Tipo: REAL, ¿Nulo?: 0, Clave Primaria: 0
  - Nombre: timestamp, Tipo: INTEGER, ¿Nulo?: 0, Clave Primaria: 0

Primeras 10 filas de la tabla 'ratings':
(1, 1, 4.0, 964982703)
(1, 3, 4.0, 964981247)
(1, 6, 4.0, 964982224)
(1, 47, 5.0, 964983815)
(1, 50, 5.0, 964982931)
(1, 70, 3.0, 964982400)
(1, 101, 5.0, 964980868)
(1, 110, 4.0, 964982176)
(1, 151, 5.0, 964984041)
(1, 157, 5.0, 964984100)

Número total de filas en 'ratings': 100836
------------

Explorando la tabla 'movies'

Esquema de la tabla 'movies':
  - Nombre: movieId, Tipo: INTEGER, ¿Nulo?: 0, Clave Primaria: 0
  - Nombre: title, Tipo: TEXT, ¿Nulo?: 0, Clave Primaria: 0
  - Nombre: genres, Tipo: TEXT, ¿Nulo?: 0, Clave Primaria: 0

Primeras 10 filas de la tabla 'movies':
(1, 'Toy Story (1995)', 'Adventure|Anim

Puesto que al realizar la exploración general (PRAGMA) las tablas 'ratings' y 'ratings2' muestran la misma información, se decide realizar una analisis mas profundo para comprar la infomación de cada fila de las tablas.

In [31]:
comparar_tablas(con, 'ratings', 'ratings2')

True

Se confirma que la información de las filas de las tablas 'ratings' y 'rating2' son iguales, por ello se procede a eliminar una de ellas y solo guardar la tabla 'ratings'

In [37]:
cur.execute("DROP TABLE ratings2;")
con.commit()
print("La tabla 'ratings2' ha sido eliminada.")

OperationalError: no such table: ratings2

## Exploración detallada

## PANDAS

In [10]:
#Crear df's de pandas con las tablas
df_ratings = pd.read_sql('select * from ratings', con)
df_movies = pd.read_sql('select * from movies', con)

#Exploración de df_ratings

In [ ]:
#Visualización de df_ratings
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
#Renombrar columnas del df e indexarlo con movie_id
df_ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
df_ratings = df_ratings.set_index('movie_id')
df_ratings

,user_id,rating,timestamp
movie_id,,,
1,1,4.0,964982703
3,1,4.0,964981247
6,1,4.0,964982224
47,1,5.0,964983815
50,1,5.0,964982931
...,...,...,...
166534,610,4.0,1493848402
168248,610,5.0,1493850091
168250,610,5.0,1494273047


In [ ]:
#La columna 'fecha_vista' es en realidad un unix timestamp, es decir, el número de segundos desde el 1 de enero de 1970 (UTC). Se restructura.
df_ratings['fecha_vista'] = pd.to_datetime(df_ratings['timestamp'], unit='s')
df_ratings

,user_id,rating,timestamp,fecha_vista
movie_id,,,,
1,1,4.0,964982703,2000-07-30 18:45:03
3,1,4.0,964981247,2000-07-30 18:20:47
6,1,4.0,964982224,2000-07-30 18:37:04
47,1,5.0,964983815,2000-07-30 19:03:35
50,1,5.0,964982931,2000-07-30 18:48:51
...,...,...,...,...
166534,610,4.0,1493848402,2017-05-03 21:53:22
168248,610,5.0,1493850091,2017-05-03 22:21:31
168250,610,5.0,1494273047,2017-05-08 19:50:47


#Exploración de df_movies

In [ ]:
#Visualización de df_movies
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
#Renombrar columnas del df e indexarlo por movie_id
df_movies.columns = ['movie_id', 'titulo', 'genero']
df_movies = df_movies.set_index('movie_id')
df_movies

,titulo,genero
movie_id,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama
